In [3]:
import sys
sys.path.append('/home/rishi/Projects/VoxelNet/VoxelNet_PyTorch/')
# from base import BaseDataSet, BaseDataLoader
import numpy as np
import os
import scipy
import torch
from PIL import Image
import cv2 
from torch.utils.data import Dataset
from torchvision import transforms

In [ ]:
class KITTIDataset(BaseDataSet):

    def __init__(self, **kwargs):
        self.data_dir = data_dir
        self.shuffle = shuffle
        sef.aug = aug
        self.is_testset = is_testset
        super(VOCDataset, self).__init__(**kwargs)

    def _set_files(self):
        self.f_rgb = glob.glob(os.path.join(self.data_dir, 'image_2', '*.png'))
        self.f_lidar = glob.glob(os.path.join(self.data_dir, 'velodyne', '*.bin'))
        self.f_label = glob.glob(os.path.join(self.data_dir, 'label_2', '*.txt'))
        
        self.f_rgb.sort()
        self.f_lidar.sort()
        self.f_label.sort()
        
        self.data_tag = [name.split('/')[-1].split('.')[-2] for name in self.f_rgb]
        
        assert len(self.data_tag) != 0, 'Dataset folder is not correct!'
        assert len(self.data_tag) == len(self.f_rgb) == len(self.f_lidar), 'Dataset folder is not correct!'
        
        nums = len(self.f_rgb)
        self.indices = list(range(nums))
        
        if self.shuffle:
            np.random.shuffle(self.indices)
            
    def _load_data(self, index, is_testset = False):
        file_id = self.indices[index]
        rgb = Image.open(self.f_rgb[file_id])
        raw_lidar = np.fromfile(self.f_lidar[file_id], dtype = np.float32).reshape((-1, 4))
        if not is_testset:
            labels = [line for line in open(self.f_label[file_id], 'r').readlines()]
        else:
            labels = ['']
        
        tag = self.data_tag[index]
        voxel = process_pointcloud(raw_lidar)
        return tag, rgb, raw_lidar, voxel, labels

In [ ]:
class KITTI(BaseDataLoader):
    def __init__(self, data_dir, batch_size, split, crop_size=None, base_size=None, scale=True, num_workers=1, val=False,
                    shuffle=False, flip=False, rotate=False, blur= False, augment=False, val_split= None, return_id=False):

        kwargs = {
            'root': data_dir,
            'split': split,
            'val': val
        }
    
        if split == "train:
            self.dataset = KITTIDataset(**kwargs)
        elif split == "val":
            self.dataset = KITTIDataset(**kwargs)
        else: raise ValueError(f"Invalid split name {split}")
        super(KITTI, self).__init__(self.dataset, batch_size, shuffle, num_workers, val_split)    